In [1]:
import lark

with open('day18_input.txt') as f:
    lines = [line.strip() for line in f.readlines()]

In [41]:
grammar = """
  ?element: element "+" atom -> sum
          | element "*" atom -> product
          | atom
  ?atom: NUMBER              -> number
       | "(" element ")"     -> group
  %import common.NUMBER
  %import common.WS_INLINE
  %ignore WS_INLINE
"""

@lark.v_args(inline=True)
class SimpleMathTransformer(lark.Transformer):
    number = float

    def __init__(self, verbose=False):
        self.verbose = verbose

    def sum(self, a, b):
        if self.verbose: print(f'{a} + {b}')
        return a + b
    
    def number(self, val):
        return int(val)
    
    def group(self, val):
        if self.verbose: print(f'group={val}')
        return val
    
    def product(self, a, b):
        if self.verbose: print(f'{a} * {b}')
        return a * b

parser = lark.Lark(grammar, parser="lalr", start="element", transformer=SimpleMathTransformer())

In [23]:
def tree_to_json_str(item):
    output = []
    tree_to_json(item, output.append)  # will build output in memory
    return ''.join(output)

def tree_to_json(item, write=None):
    """ Writes a Lark tree as a JSON dictionary. """
    if write is None: write = sys.stdout.write
    _tree_to_json(item, write, 0)

def _tree_to_json(item, write, level):
    indent = '  ' * level
    level += 1
    if isinstance(item, lark.Tree):
        write(f'{indent}{{ "type": "{item.data}", "children": [\n')
        sep = ''
        for child in item.children:
            write(indent)
            write(sep)
            _tree_to_json(child, write, level)
            sep = ',\n'
        write(f'{indent}] }}\n')
    elif isinstance(item, lark.Token):
        # reminder: Lark Tokens are directly strings
        # token attrs include: line, end_line, column, end_column, pos_in_stream, end_pos
        write(f'{indent}{{ "type": "{item.type}", "text": "{item}", "line": {item.line}, "col": {item.column} }}\n')
    else:
        assert False, item  # fall-through
    
with open("C:\\Users\\westo\\Downloads\\ast.json", "w") as f:
    tree_to_json(ast, f.write)

In [34]:
sum = 0
for expression in lines:
    sum += parser.parse(expression)
print(sum)

12918250417632


In [54]:
grammar = """
  ?product: sum
          | product "*" sum -> product
  ?sum:     atom
          | sum "+" atom     -> sum
  ?atom:    NUMBER           -> number
          | "(" product ")"  -> group
  %import common.NUMBER
  %import common.WS_INLINE
  %ignore WS_INLINE
"""
parser = lark.Lark(grammar, parser="lalr", start="product", transformer=SimpleMathTransformer(verbose=True))
print(lines[0])
parser.parse(lines[0])

(5 * (5 * 7 * 6 * 7 + 2) * 5 + 7) + 3 * 2 * 8 + 8
5 * 7
35 * 6
7 + 2
210 * 9
group=1890
5 * 1890
5 + 7
9450 * 12
group=113400
113400 + 3
113403 * 2
8 + 8
226806 * 16


3628896